# Run fMRIPrep for a single subject (1001)

This notebook builds and optionally runs an fMRIPrep command on the BIDS dataset `ds004920-main` for participant `1001`.

Requirements:
- fmriprep installed and on PATH (or available via Docker/Singularity)
- FreeSurfer license if you want surface reconstruction (set FS_LICENSE env var or provide path below)

The notebook will construct a safe command, show it, and ask for confirmation before running.

In [18]:
# Cell 2: Configure paths and parameters
import os
import shutil
import subprocess
import sys
import multiprocessing

# Adjust these paths if needed
bids_dir = '/home/jovyan/Desktop/ds004920-main'
out_dir = os.path.join(bids_dir, 'derivatives', 'fmriprep')
work_dir = os.path.join(bids_dir, 'work', 'fmriprep')
participant_label = '1001'

# Use the FreeSurfer license in the working directory
fs_license = os.path.abspath('license.txt')

print('BIDS directory:', bids_dir)
print('Output directory:', out_dir)
print('Work directory:', work_dir)
print('Participant label:', participant_label)
print('FreeSurfer license candidate:', fs_license)

BIDS directory: /home/jovyan/Desktop/ds004920-main
Output directory: /home/jovyan/Desktop/ds004920-main/derivatives/fmriprep
Work directory: /home/jovyan/Desktop/ds004920-main/work/fmriprep
Participant label: 1001
FreeSurfer license candidate: /home/jovyan/Desktop/license.txt


In [19]:
# Cell 3: Basic checks (no BIDS directory check needed)
# Only create output and work directories if missing
os.makedirs(out_dir, exist_ok=True)
os.makedirs(work_dir, exist_ok=True)

# quick sanity check for participant folder
sub_folder = os.path.join(bids_dir, f'sub-{participant_label}')
if not os.path.isdir(sub_folder):
    print(f"Warning: subject folder not found at {sub_folder}. Check subject label or dataset.")
else:
    print(f"Found subject folder: {sub_folder}")

In [20]:
# Cell 4: Check for fmriprep binary
fmriprep_bin = shutil.which('fmriprep')
if fmriprep_bin is None:
    print("fmriprep not found on PATH.")
    print("Options:\n - Install with pip: pip install fmriprep\n - Use the Docker image: docker run --rm -it ... fmriprep\n - Use Singularity/Apptainer on HPC.")
    print("If you plan to run fmriprep from this notebook, install it or run the Docker/Singularity command from a terminal.")
else:
    print('fmriprep found at:', fmriprep_bin)

fmriprep not found on PATH.
Options:
 - Install with pip: pip install fmriprep
 - Use the Docker image: docker run --rm -it ... fmriprep
 - Use Singularity/Apptainer on HPC.
If you plan to run fmriprep from this notebook, install it or run the Docker/Singularity command from a terminal.


In [21]:
# Cell 5: Build the fmriprep command
cpus = max(1, multiprocessing.cpu_count() // 2)
mem_mb = 16000  # adjust to available memory on your machine

cmd = [
    fmriprep_bin or 'fmriprep',
    bids_dir,
    out_dir,
    'participant',
    '--participant-label', participant_label,
    '--n_cpus', str(cpus),
    '--mem_mb', str(mem_mb),
    '--work-dir', work_dir,
    '--output-spaces', 'MNI152NLin2009cAsym:res-2'
]

# Add FreeSurfer license or skip recon-all
if os.path.isfile(fs_license):
    cmd += ['--fs-license-file', fs_license]
    print(f'Using FreeSurfer license at: {fs_license}')
else:
    cmd += ['--fs-no-reconall']
    print('FreeSurfer license not found; recon-all will be skipped (surface outputs unavailable).')

# Example safety flags: do not run heavy BIDS-Derivatives rewriting
cmd += ['--skip-bids-validation']

print('Constructed fmriprep command:')
print(' '.join(cmd))

FreeSurfer license not found; recon-all will be skipped (surface outputs unavailable).
Constructed fmriprep command:
fmriprep /home/jovyan/Desktop/ds004920-main /home/jovyan/Desktop/ds004920-main/derivatives/fmriprep participant --participant-label 1001 --n_cpus 4 --mem_mb 16000 --work-dir /home/jovyan/Desktop/ds004920-main/work/fmriprep --output-spaces MNI152NLin2009cAsym:res-2 --fs-no-reconall --skip-bids-validation


In [22]:
# Cell 6: Ask user and run the command
run_now = input('Run fmriprep now for participant ' + participant_label + '? (y/N): ').strip().lower()
if run_now == 'y':
    print('Running fmriprep... this may take a long time. Output will stream below.')
    # Run and stream output
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    try:
        for line in process.stdout:
            print(line, end='')
    except KeyboardInterrupt:
        process.terminate()
        print('\nProcess terminated by user.')
    ret = process.wait()
    print('fmriprep finished with return code', ret)
else:
    print('Aborted. To run later, set run_now="y" or run the printed command in a terminal.')

Running fmriprep... this may take a long time. Output will stream below.


FileNotFoundError: [Errno 2] No such file or directory: 'fmriprep'

Notes:
- Running fMRIPrep inside a notebook can be done but is often better run from a terminal or job scheduler (Docker/Singularity) because it uses many resources.
- If you will use FreeSurfer surfaces, place your license at the path above or set FS_LICENSE in your environment.
- Adjust CPU and memory settings to match your environment.